# CNN1

Training with the persistence image of the PCA Fourier Series of a signal

In [1]:
import numpy as np
import pandas as pd

### Load the data

In [2]:
#load list imagenes as numpy array
imagenes=np.load('data/imagenes.npy', allow_pickle=True)
labels = np.load("data/labels.npy")
lol=[]
for i in range(len(imagenes)):
    lol.append(imagenes[i].flatten())
imagenes=np.array(lol)
imagenes.shape

(1500, 25000)

In [3]:


X = imagenes

y=labels

# Reshape the data to add a channel dimension
X = X.reshape(-1, imagenes.shape[1], 1)


# CNN Architecture

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Define the model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(imagenes.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)), #This part can be modified to stop overfitting
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


d:\anaconda3_5\envs\gcwtda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 24998, 32)      │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 12499, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12499, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 12497, 64)      │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 6248, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6248, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 399872)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    51,183,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,190,209 (195.28 MB)

 Trainable params: 51,190,209 (195.28 MB)

 Non-trainable params: 0 (0.00 B)

# Training

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_val, y_val))


AttributeError: module 'ml_dtypes' has no attribute 'float4_e2m1fn'
Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 49s 5s/step - accuracy: 0.5093 - loss: 1.7946 - val_accuracy: 0.6733 - val_loss: 1.0021
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.6502 - loss: 0.8862 - val_accuracy: 0.5033 - val_loss: 0.8153
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step - accuracy: 0.5673 - loss: 0.7778 - val_accuracy: 0.6667 - val_loss: 0.7362
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 41s 4s/step - accuracy: 0.6327 - loss: 0.7213 - val_accuracy: 0.6800 - val_loss: 0.7059
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step - accuracy: 0.6689 - loss: 0.6977 - val_accuracy: 0.6700 - val_loss: 0.6896


In [6]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')


10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - accuracy: 0.6817 - loss: 0.6865
Validation Loss: 0.689571738243103, Validation Accuracy: 0.6700000166893005


Usando *R= 0.4* el modelo logra 0.73 de accuracy en el set de validación

In [7]:
# Save the model
model.save('data/cnn1_model_0.4.keras')